# Car Prediction #
İkinci el araç fiyatlarını (özelliklerine göre) tahmin eden modeller oluşturma ve MLOPs ile Hugging Face üzerinden yayımlayacağız.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split #veri setini bölme işlemleri
from sklearn.linear_model import LinearRegression #Doğrusal regresyon
from sklearn.metrics import r2_score,mean_squared_error #modelimizin performansını ölçmek için
from sklearn.compose import ColumnTransformer #Sütun dönüşüm işlemleri
from sklearn.preprocessing import OneHotEncoder, StandardScaler # kategori - sayısal dönüşüm  ve ölçeklendirme
from sklearn.pipeline import Pipeline #Veri işleme hattı

In [ ]:
!pip instal xlrd

In [ ]:
!pip install streamlit

In [ ]:
#Excell dosyalarını okumak için

In [2]:
!pip install xldr

Defaulting to user installation because normal site-packages is not writeable

ERROR: Could not find a version that satisfies the requirement xldr (from versions: none)
ERROR: No matching distribution found for xldr


## Veri dosyasını yükle

In [3]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 4821-D018

 Directory of C:\Users\emura\Documents\Techist\ders 10

11.06.2024  18:47    <DIR>          .
11.06.2024  17:48    <DIR>          ..
11.06.2024  18:25    <DIR>          .ipynb_checkpoints
11.06.2024  18:47             1.338 car_predict_tech.ipynb
22.05.2024  20:43           141.824 cars.xls
               2 File(s)        143.162 bytes
               3 Dir(s)  15.865.368.576 bytes free


In [5]:
df=pd.read_excel('cars.xls')
df

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.103129,8221,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,1
1,17542.036083,9135,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
2,16218.847862,13196,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
3,16336.913140,16342,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,0
4,16339.170324,19832,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
799,16507.070267,16229,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,0,0
800,16175.957604,19095,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0
801,15731.132897,20484,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0
802,15118.893228,25979,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Price     804 non-null    float64
 1   Mileage   804 non-null    int64  
 2   Make      804 non-null    object 
 3   Model     804 non-null    object 
 4   Trim      804 non-null    object 
 5   Type      804 non-null    object 
 6   Cylinder  804 non-null    int64  
 7   Liter     804 non-null    float64
 8   Doors     804 non-null    int64  
 9   Cruise    804 non-null    int64  
 10  Sound     804 non-null    int64  
 11  Leather   804 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 75.5+ KB


In [6]:
# Veri ön işleme

In [7]:
X=df.drop('Price',axis=1) #fiyat sütunu çıkar fiyata etki edenler kalsın
y=df['Price'] #tahmin

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

####  Veri ön işleme, standartlaştırma ve OHE işlemlerini otomatikleştiriyoruz (standarlaştırıyoruz). Artık preprocess kullanarak kullanıcında arayüz aracılığıyla gelen veriyi mdoelimize uygun hale çevirebiliriz.


In [11]:
preprocess=ColumnTransformer(
    transformers=[
        ('num',StandardScaler(),['Mileage', 'Cylinder','Liter','Doors']),
        ('cat',OneHotEncoder(),['Make','Model','Trim','Type'])
    ]
)

In [12]:
my_model=LinearRegression()

In [13]:
#pipeline ı tanımla
pipe=Pipeline(steps=[('preprocessor',preprocess),('model',my_model)])

In [14]:
#pipeline fit
pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Mileage', 'Cylinder',
                                                   'Liter', 'Doors']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Make', 'Model', 'Trim',
                                                   'Type'])])),
                ('model', LinearRegression())])

In [16]:
y_pred=pipe.predict(X_test)
print('RMSE',mean_squared_error(y_test,y_pred)**0.5)
print('R2',r2_score(y_test,y_pred))

RMSE 835.100716728648
R2 0.9912072828879327


In [ ]:
#isterseniz veri setinin tammamıyla tekrar eğitim yapabilirsiniz.
#pipe.fit(X,y)

## Streamlit ile modeli yayma/deploy/kullanıma sunma

In [17]:
!pip install streamlit

Defaulting to user installation because normal site-packages is not writeable


In [18]:
df['Mileage'].max()

50387

In [19]:
df['Type'].unique()

array(['Sedan', 'Convertible', 'Hatchback', 'Coupe', 'Wagon'],
      dtype=object)

In [20]:
df['Liter'].max()

6.0

#### Python ile yapılan çalışmnalrın hızlı bir şekilde deploy edilmesi için HTML render arayüzler tasarlamanızı sağlar.

In [21]:
import streamlit as st
#price tahmin fonksiyonu tanımla
def price(make,model,trim,mileage,car_type,cylinder,liter,doors,cruise,sound,leather):
    input_data=pd.DataFrame({'Make':[make],
                             'Model':[model],
                             'Trim':[trim],
                             'Mileage':[mileage],
                             'Type':[car_type],
                             'Cylinder':[cylinder],
                             'Liter':[liter],
                             'Doors':[doors],
                             'Cruise':[cruise],
                             'Sound':[sound],
                             'Leather':[leather]})
    prediction=pipe.predict(input_data)[0]
    return prediction
st.title("II. El Araba Fiyatı Tahmin:red_car: @drmurataltun")
st.write('Arabanın özelliklerini seçiniz')
make=st.selectbox('Marka',df['Make'].unique())
model=st.selectbox('Model',df[df['Make']==make]['Model'].unique())
trim=st.selectbox('Trim',df[(df['Make']==make) &(df['Model']==model)]['Trim'].unique())
mileage=st.number_input('Kilometre',100,200000)
car_type=st.selectbox('Araç Tipi',df[(df['Make']==make) &(df['Model']==model)&(df['Trim']==trim)]['Type'].unique())
cylinder=st.selectbox('Cylinder',df['Cylinder'].unique())
liter=st.number_input('Yakıt hacmi',1,10)
doors=st.selectbox('Kapı sayısı',df['Doors'].unique())
cruise=st.radio('Hız Sbt.',[True,False])
sound=st.radio('Ses Sis.',[True,False])
leather=st.radio('Deri döşeme.',[True,False])
if st.button('Tahmin'):
    pred=price(make,model,trim,mileage,car_type,cylinder,liter,doors,cruise,sound,leather)
    st.write('Fiyat:$', round(pred[0],2))

2024-06-11 20:08:59.487 
  command:

    streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [25]:
#streamlit run C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py